In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

# reflect the tables


In [11]:
Base.classes.keys()
# Map the Measurement Class
MEASUREMENT = Base.classes.measurement
# Map the Station Class
STATION = Base.classes.station

session = Session(engine)


In [7]:
# Use inspector to explore the columns
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [8]:
for c in inspector.get_columns("measurement"):
    print(c["name"],c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [9]:
for c in inspector.get_columns("station"):
    print(c["name"],c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [14]:
measures = session.query(MEASUREMENT).all()
stations= session.query(STATION).all()

In [16]:
session.query(MEASUREMENT).count()


19550

In [17]:
session.query(STATION).count()

9

In [ ]:
# We can view all of the classes that automap found  ***CHECK


In [ ]:
# Save references to each table     ***CHECK


In [21]:
# Create our session (link) from Python to the DB
engine.execute('select * from measurement limit 5').fetchall()

pd.read_sql('select * from measurement limit 5',engine)

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0


In [18]:
pd.read_sql('select * from station limit 10',engine)

,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,4,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,6,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,7,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,8,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,9,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [20]:
session.query(MEASUREMENT.date).order_by(MEASUREMENT.date).all()

[('2010-01-01'),
 ('2010-01-01'),
 ('2010-01-01'),
 ('2010-01-01'),
 ('2010-01-01'),
 ('2010-01-01'),
 ('2010-01-01'),
 ('2010-01-02'),
 ('2010-01-02'),
 ('2010-01-02'),
 ('2010-01-02'),
 ('2010-01-02'),
 ('2010-01-02'),
 ('2010-01-02'),
 ('2010-01-03'),
 ('2010-01-03'),
 ('2010-01-03'),
 ('2010-01-03'),
 ('2010-01-03'),
 ('2010-01-03'),
 ('2010-01-03'),
 ('2010-01-04'),
 ('2010-01-04'),
 ('2010-01-04'),
 ('2010-01-04'),
 ('2010-01-04'),
 ('2010-01-04'),
 ('2010-01-04'),
 ('2010-01-05'),
 ('2010-01-05'),
 ('2010-01-05'),
 ('2010-01-05'),
 ('2010-01-05'),
 ('2010-01-05'),
 ('2010-01-06'),
 ('2010-01-06'),
 ('2010-01-06'),
 ('2010-01-06'),
 ('2010-01-06'),
 ('2010-01-06'),
 ('2010-01-07'),
 ('2010-01-07'),
 ('2010-01-07'),
 ('2010-01-07'),
 ('2010-01-07'),
 ('2010-01-07'),
 ('2010-01-07'),
 ('2010-01-08'),
 ('2010-01-08'),
 ('2010-01-08'),
 ('2010-01-08'),
 ('2010-01-08'),
 ('2010-01-08'),
 ('2010-01-08'),
 ('2010-01-09'),
 ('2010-01-09'),
 ('2010-01-09'),
 ('2010-01-09'),
 ('2010-01-09'

In [30]:
# Find all the dates greater than 2011-03-01
(
    session
    .query(MEASUREMENT.date)
    .filter(MEASUREMENT.date>= '2010-01-01')
    .filter(MEASUREMENT.date<='2010-12-01')
    .order_by(MEASUREMENT.date).count()
)


2557

In [29]:
(
    session
    .query(MEASUREMENT.date)
    .filter(MEASUREMENT.date.between('2010-01-01', '2010-12-01'))
    .order_by(MEASUREMENT.date).all()
)

2557

In [33]:
pd.read_sql('select prcp,date from measurement where date between ('2010-01-01','2010-12-01')',engine)

SyntaxError: invalid syntax (<ipython-input-33-90aa9f2e3108>, line 1)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date  

# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
